# Kernel Semântico com Integração do Servidor OpenBnB MCP

Este notebook demonstra como utilizar o Kernel Semântico com o servidor OpenBnB MCP real para pesquisar alojamentos reais do Airbnb usando o MCPStdioPlugin. Para acesso ao LLM, utiliza o Azure AI Foundry. Para configurar as suas variáveis de ambiente, pode seguir a [Lição de Configuração](/00-course-setup/README.md)


## Importar os Pacotes Necessários


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Criar a Conexão do Plugin MCP

Vamos conectar ao [servidor MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) utilizando o MCPStdioPlugin. Este servidor oferece funcionalidades de pesquisa do Airbnb através do pacote @openbnb/mcp-server-airbnb.


## Criar o Cliente

Neste exemplo, vamos utilizar o Azure AI Foundry para aceder ao LLM. Certifique-se de que as variáveis de ambiente estão configuradas corretamente.


## Configuração do Ambiente

Configure as definições do Azure OpenAI. Certifique-se de que tem as seguintes variáveis de ambiente configuradas:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Compreender a Integração do OpenBnB MCP

Este notebook conecta-se ao **servidor real OpenBnB MCP**, que fornece funcionalidade de pesquisa real do Airbnb.

### Como funciona:

1. **MCPStdioPlugin**: Utiliza comunicação de entrada/saída padrão com o servidor MCP  
2. **Pacote NPM Real**: Faz o download e executa `@openbnb/mcp-server-airbnb` via npx  
3. **Dados em Tempo Real**: Retorna dados reais de propriedades do Airbnb a partir das suas APIs  
4. **Descoberta de Funções**: O agente descobre automaticamente as funções disponíveis no servidor MCP  

### Funções Disponíveis:

O servidor OpenBnB MCP normalmente fornece:  
- **search_listings** - Pesquisar propriedades do Airbnb por localização e critérios  
- **get_listing_details** - Obter informações detalhadas sobre propriedades específicas  
- **check_availability** - Verificar disponibilidade para datas específicas  
- **get_reviews** - Obter avaliações de propriedades  
- **get_host_info** - Obter informações sobre os anfitriões das propriedades  

### Pré-requisitos:

- **Node.js** instalado no seu sistema  
- **Ligação à Internet** para fazer o download do pacote do servidor MCP  
- **NPX** disponível (vem incluído com o Node.js)  

### Testar a Conexão:

Pode testar o servidor MCP manualmente executando:  
```bash
npx -y @openbnb/mcp-server-airbnb
```

Isto fará o download e iniciará o servidor OpenBnB MCP, ao qual o Semantic Kernel se conecta para obter dados reais do Airbnb.


## Executar o Agente com o Servidor OpenBnB MCP

Agora vamos executar o Agente de IA que se conecta ao servidor OpenBnB MCP para procurar alojamentos reais no Airbnb em Estocolmo para 2 adultos e 1 criança. Sinta-se à vontade para alterar a lista `user_inputs` para modificar os critérios de pesquisa.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Resumo  
Parabéns! Conseguiu criar um agente de IA que se integra com a pesquisa de alojamentos reais utilizando o Model Context Protocol (MCP):  

Tecnologias Utilizadas:  
Semantic Kernel - Para criar agentes inteligentes com Azure OpenAI  
Azure AI Foundry - Para capacidades de LLM e conclusão de chat  
MCP (Model Context Protocol) - Para integração padronizada de ferramentas  
OpenBnB MCP Server - Para funcionalidade real de pesquisa no Airbnb  
Node.js/NPX - Para executar o servidor MCP externo  

O Que Aprendeu:  
Integração MCP: Conectar agentes do Semantic Kernel a servidores MCP externos  
Acesso a Dados em Tempo Real: Pesquisar propriedades reais do Airbnb através de APIs ao vivo  
Comunicação por Protocolo: Utilizar comunicação stdio entre o agente e o servidor MCP  
Descoberta de Funções: Descobrir automaticamente funções disponíveis nos servidores MCP  
Respostas em Streaming: Capturar e registar chamadas de funções em tempo real  
Renderização em HTML: Formatar respostas do agente com tabelas estilizadas e exibições interativas  

Próximos Passos:  
Integrar servidores MCP adicionais (meteorologia, voos, restaurantes)  
Construir um sistema multiagente combinando protocolos MCP e A2A  
Criar servidores MCP personalizados para as suas próprias fontes de dados  
Implementar memória de conversação persistente entre sessões  
Desenvolver o agente em Azure Functions com orquestração de servidores MCP  
Adicionar autenticação de utilizadores e capacidades de reserva  

Principais Vantagens da Arquitetura MCP:  
Padronização: Protocolo universal para conectar agentes de IA a ferramentas externas  
Dados em Tempo Real: Acesso a informações atualizadas e ao vivo de vários serviços  
Extensibilidade: Integração fácil de novas fontes de dados e ferramentas  
Interoperabilidade: Funciona em diferentes frameworks de IA e plataformas de agentes  
Separação de Preocupações: Distinção clara entre lógica de IA e acesso a dados externos  



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante ter em conta que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
